# Векторизованные операции над строками

Одна из сильных сторон языка Python — относительное удобство работы в нем со строковыми данными и манипуляций ими. Библиотека Pandas вносит в это свою лепту и предоставляет набор векторизованных операций над строками, ставших существенной частью очистки данных, необходимой при работе с реальными данными. В этом разделе мы изучим некоторые строковые операции библиотеки Pandas, после чего рассмотрим их использование для частичной очистки очень зашумленного набора данных рецептов, собранных в Интернете.

## Знакомство со строковыми операциями библиотеки Pandas

В предыдущем разделе мы видели, как обобщают арифметические операции такие инструменты, как библиотека NumPy и библиотека Pandas, позволяя легко и быстро выполнять одну и ту же операцию над множеством элементов массива. Например:

In [1]:
import numpy as np
x = np.array([2, 3, 5, 7, 11, 13])
x * 2

array([ 4,  6, 10, 14, 22, 26])

Векторизация операций упрощает синтаксис работы с массивами данных: больше нет необходимости беспокоиться о размере или форме массива, а только о нужной нам операции. Библиотека NumPy не предоставляет такого простого способа доступа для массивов строк, так что приходится использовать более длинный синтаксис циклов:

In [2]:
data = ['peter', 'Paul', 'MARY', 'gUIDO']
[s.capitalize() for s in data]

['Peter', 'Paul', 'Mary', 'Guido']

Вероятно, для работы с некоторыми данными этого достаточно, но при наличии отсутствующих значений все портится. Например:

In [3]:
#data = ['peter', 'Paul', 'MARY', 'gUIDO', None]
#[s.capitalize() for s in data]

Библиотека Pandas включает средства как для работы с векторизованными строковыми операциями, так и для корректной обработки отсутствующих значений посредством атрибута str объектов Series библиотеки Pandas и содержащих строки объектов Index. Так, допустим, мы создали объект Series библиотеки Pandas с теми же данными:

In [4]:
import pandas as pd
names = pd.Series(data)
names

0    peter
1     Paul
2     MARY
3    gUIDO
dtype: object

Теперь можно вызвать один-единственный метод для преобразования строчных букв в заглавные, который будет игнорировать любые отсутствующие значения:

In [5]:
names.str.capitalize()

0    Peter
1     Paul
2     Mary
3    Guido
dtype: object

## Таблицы методов работы со строками библиотеки Pandas

Если вы хорошо разбираетесь в манипуляции строковыми данными в языке Python, львиная доля синтаксиса работы со строками библиотеки Pandas будет вам интуитивно понятна настолько, что достаточно, наверное, просто привести таблицу имеющихся методов. С этого и начнем, прежде чем углубимся в некоторые нюансы. Примеры в этом разделе используют следующий ряд имен:

In [6]:
monte = pd.Series(['Graham Chapman', 'John Cleese', 'Terry Gilliam', 'Eric Idle', 'Terry Jones', 'Michael Palin'])

### Методы, аналогичные строковым методам языка Python

Практически для всех встроенных строковых методов Python есть соответствующий векторизованный строковый метод библиотеки Pandas. Вот список методов атрибута str библиотеки Pandas, дублирующий строковые методы языка Python:
len() lower() translate() islower()
ljust() upper() startswith() isupper()
rjust() find() endswith() isnumeric()
center() rfind() isalnum() isdecimal()
zfill() index() isalpha() split()
strip() rindex() isdigit() rsplit()
rstrip() capitalize() isspace() partition()
lstrip() swapcase() istitle() rpartition()

Обратите внимание, что возвращаемые значения у них отличаются. Некоторые, например lower(), возвращают Series строк:

In [7]:
monte.str.lower()

0    graham chapman
1       john cleese
2     terry gilliam
3         eric idle
4       terry jones
5     michael palin
dtype: object

Часть других возвращает числовые значения:

In [8]:
monte.str.len()

0    14
1    11
2    13
3     9
4    11
5    13
dtype: int64

Или булевы значения:

In [9]:
monte.str.startswith('T')

0    False
1    False
2     True
3    False
4     True
5    False
dtype: bool

Или списки и другие составные значения для каждого элемента:

In [10]:
monte.str.split()

0    [Graham, Chapman]
1       [John, Cleese]
2     [Terry, Gilliam]
3         [Eric, Idle]
4       [Terry, Jones]
5     [Michael, Palin]
dtype: object

### Методы, использующие регулярные выражения

Помимо этого, существует и несколько методов, принимающих на входе регулярные выражения для проверки содержимого каждого из строковых элементов и следующих некоторым соглашениям по API встроенного модуля re языка Python

Метод
Описание
match()
Вызывает функцию re.match() для каждого элемента, возвращая булево значение
extract()
Вызывает функцию re.match() для каждого элемента, возвращая подходящие группы в виде строк
findall()
Вызывает функцию re.findall() для каждого элемента
replace()
Заменяет вхождения шаблона какой-либо другой строкой
contains()
Вызывает функцию re.search() для каждого элемента, возвращая булево значение
count()
Подсчитывает вхождения шаблона
split()
Эквивалент функции str.split(), но принимающий на входе регулярные выражения
rsplit()
Эквивалент функции str.rsplit(), но принимающий на входе регулярные выражения

С помощью этих функций можно выполнять массу интересных операций. Например, можно извлечь имя из каждого элемента, выполнив поиск непрерывной группы символов в начале каждого из них:

In [11]:
monte.str.extract('([A-Za-z]+)')

,0
0,Graham
1,John
2,Terry
3,Eric
4,Terry
5,Michael


Или, например, найти все имена, начинающиеся и заканчивающиеся согласным звуком, воспользовавшись символами регулярных выражений «начало строки» (^) и «конец строки» ($):

In [12]:
monte.str.findall(r'^[^AEIOU].*[^aeiou]$')

0    [Graham Chapman]
1                  []
2     [Terry Gilliam]
3                  []
4       [Terry Jones]
5     [Michael Palin]
dtype: object

Такой сжатый синтаксис регулярных выражений для записей объектов Series и DataFrame открывает массу возможностей для анализа и очистки данных.

### Прочие методы

Наконец, существуют и прочие методы, пригодные для разных удобных операций

#### Векторизированный доступ к элементам и вырезание подстрок.

Операции get() и slice(), в частности, предоставляют возможность векторизованного доступа к элементам из каждого массива. Например, можно вырезать первые три символа из каждого массива посредством выражения str.slice(0, 3). Обратите внимание, что такая возможность доступна и с помощью обычного синтаксиса индексации языка Python, например, df.str.slice(0, 3) эквивалентно df.str[0:3]:

In [13]:
monte.str[0:3]

0    Gra
1    Joh
2    Ter
3    Eri
4    Ter
5    Mic
dtype: object

Индексация посредством df.str.get(i) и df.str[i] происходит аналогично.

Эти методы get() и slice() также дают возможность обращаться к элементам возвращаемых методом split() массивов. Например, для извлечения фамилии из каждой записи можно использовать вместе методы split() и get():

In [14]:
monte.str.split().str.get(-1)

0    Chapman
1     Cleese
2    Gilliam
3       Idle
4      Jones
5      Palin
dtype: object

In [15]:
monte.str.split().str[0]

0     Graham
1       John
2      Terry
3       Eric
4      Terry
5    Michael
dtype: object

#### Индикаторные переменные

Еще один метод, требующий некоторых дополнительных пояснений, — get_dummies(). Удобно, когда в данных имеется столбец, содержащий кодированный индикатор. Например, у нас есть набор данных, содержащий информацию в виде кодов, таких как A="родился в США", B="родился в Великобритании", C="любит сыр", D="любит мясные консервы":

In [16]:
full_monte = pd.DataFrame({'name': monte, 'info': ['B|C|D', 'B|D', 'A|C', 'B|D', 'B|C', 'B|C|D']})
full_monte

,name,info
0,Graham Chapman,B|C|D
1,John Cleese,B|D
2,Terry Gilliam,A|C
3,Eric Idle,B|D
4,Terry Jones,B|C
5,Michael Palin,B|C|D


Метод get_dummies() дает возможность быстро разбить все индикаторные переменные, преобразовав их в объект DataFrame:

In [17]:
full_monte['info'].str.get_dummies('|')

,A,B,C,D
0,0,1,1,1
1,0,1,0,1
2,1,0,1,0
3,0,1,0,1
4,0,1,1,0
5,0,1,1,1


Используя эти операции как «строительные блоки», можно создать бесчисленное множество обрабатывающих строки процедур для очистки данных.
Мы не будем углубляться в эти методы, но я рекомендую прочитать раздел Working with Text Data («Работа с текстовыми данными») из онлайн-документации библиотекиPandas (http://pandas.pydata.org/pandas-docs/stable/text.html) или заглянуть в раздел «Дополнительные источники информации» данной главы.

## Пример: база данных рецептов

Описанные векторизованные строковые операции оказываются наиболее полезными при очистке сильно зашумленных реальных данных. Здесь мы рассмотрим пример такой очистки, воспользовавшись полученной из множества различных интернет-источников базой данных рецептов. Наша цель — разбор рецептов на списки ингредиентов, чтобы можно было быстро найти рецепт, исходя из имеющихся в распоряжении ингредиентов.

Используемые для компиляции сценарии можно найти по адресу https://github.com/fictivekin/openrecipes, как и ссылку на актуальную версию базы.

//openrecipes.s3.amazonaws.com/20131812-recipeitems.json.gz

База данных находится в формате JSON, так что можно попробовать воспользоваться функцией pd.read_json для ее чтения:

In [19]:
try:
    recipes = pd.read_json(r'JSON\recipeitems-latest.json')
except ValueError as e:
    print('ValueError:', e)

ValueError: Trailing data


Мы получили ошибку ValueError с упоминанием наличия «хвостовых данных». Если поискать эту ошибку в Интернете, складывается впечатление, что она появляется из-за использования файла, в котором каждая строка сама по себе является корректным JSON, а весь файл в целом — нет. Рассмотрим, справедливо ли это объяснение:

In [20]:
with open(r'JSON\recipeitems-latest.json') as f:
    line = f.readline()
pd.read_json(line).shape

(2, 12)

Да, очевидно, каждая строка — корректный JSON, так что нам нужно соединить их все воедино. Один из способов сделать это — фактически сформировать строковое представление, содержащее все записи JSON, после чего загрузить все с помощью pd.read_json:

In [21]:
# read the entire file into a Python array
with open(r'JSON\recipeitems-latest.json', 'r') as f:
    # Extract each line
    data = (line.strip() for line in f)
    # Reformat so each line is the element of a list
    data_json = "[{0}]".format(','.join(data))
# read the result as a JSON
recipes = pd.read_json(data_json)

UnicodeDecodeError: 'charmap' codec can't decode byte 0x98 in position 5033: character maps to <undefined>